In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import requests
import warnings
import json
# import matplotlib.pyplot as plt
#import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

# Read in Excel Data

In [2]:
excel_path = '/'.join(['.','Data',
                      'NBA Stats_2021-2023_01292024.xlsx'])

print('Reading in games...')
games = pd.read_excel(excel_path,
                     sheet_name = 'Games',
                     header = 0,
                     index_col = 'game_id',
                     parse_dates = ['game_date'],
                     engine='openpyxl')

print('Reading in players...')
players = pd.read_excel(excel_path,
                     sheet_name = 'Players',
                     header = 0,
                       index_col = 'player_id',
                       engine='openpyxl')

print('Reading in teams...')
teams = pd.read_excel(excel_path,
                     sheet_name = 'Teams',
                     header = 0,
                     index_col = 'team_id',
                     engine='openpyxl')

print('Reading in stats...')
stats = pd.read_excel(excel_path,
                     sheet_name = 'Stats',
                     header = 0,
                     engine='openpyxl')

Reading in games...
Reading in players...
Reading in teams...
Reading in stats...


In [3]:
games.head()

,game_date,game_season,game_status,game_period,game_time,game_home_team_id,game_home_team_score,game_visitor_team_id,game_visitor_team_score,game_preseason,game_postseason
game_id,,,,,,,,,,,
473410,2021-10-19,2021,Final,4,NaN,17,127,3,104,N,False
473409,2021-10-19,2021,Final,4,NaN,14,114,10,121,N,False
473413,2021-10-20,2021,Final,4,NaN,28,83,30,98,N,False
473415,2021-10-20,2021,Final,4,NaN,15,132,6,121,N,False
473414,2021-10-20,2021,Final,6,NaN,20,138,2,134,N,False


In [4]:
players.head()

,player_team_id,player_position,player_full_name,player_first_name,player_last_name,player_height_feet,player_height_inches,player_weight_pounds
player_id,,,,,,,,
17896049,3,G,David Duke Jr.,David,Duke Jr.,NaN,NaN,NaN
17553995,14,G,Austin Reaves,Austin,Reaves,NaN,NaN,NaN
666543,14,F,Sekou Doumbouya,Sekou,Doumbouya,NaN,NaN,NaN
59,14,G,Avery Bradley,Avery,Bradley,6.0,2.0,180.0
220,14,C,Dwight Howard,Dwight,Howard,6.0,11.0,265.0


In [5]:
stats.head()

,id,game_id,player_id,team_id,opponent_team_id,min,fga,fgm,fg_pct,fta,ftm,ft_pct,fg3a,fg3m,fg3_pct,oreb,dreb,reb,pts,ast,stl,blk,pf,turnover
0,7315088,473410,17896049,3,17,0:00,0,0,0.000,0,0,0.00,0,0,0.000,0,0,0,0,0,0,0,0,0
1,7315614,473409,17553995,14,10,0:00,0,0,0.000,0,0,0.00,0,0,0.000,0,0,0,0,0,0,0,0,0
2,7315611,473409,666543,14,10,0:00,0,0,0.000,0,0,0.00,0,0,0.000,0,0,0,0,0,0,0,0,0
3,7315610,473409,59,14,10,8,3,2,0.667,0,0,NaN,3,2,0.667,1,0,1,6,1,0,0,2,1
4,7315612,473409,220,14,10,13,2,1,0.500,4,3,0.75,0,0,NaN,0,6,6,5,0,0,0,2,2


In [6]:
# Convert 'min' to numeric and fill NaN with 0
stats['min'] = pd.to_numeric(stats['min'],
                             errors = 'coerce')\
                .fillna(0)

In [7]:
# Fantasy points (PrizePicks)
## Points = 1
## Rebound = 1.2
## Assists = 1.5
## Block = 3
## Steals = 3
## Turnover = -1
stats['fpts'] = stats['pts']\
                + (1.2 * stats['reb'])\
                + (1.5 * stats['ast'])\
                + (3 * stats['blk'])\
                + (3 * stats['stl'])\
                + (-1 * stats['turnover'])

In [8]:
# Merge date of games
stats_dates = pd.merge(stats,
                      games[['game_date']],
                      how = 'left',
                      left_on = 'game_id',
                      right_index = True)\
                .sort_values(['game_date','player_id'])

In [9]:
# Merge player positions
stats_dates_pos = pd.merge(stats_dates,
                           players[['player_position']],
                           how = 'left',
                           left_on = 'player_id',
                           right_index = True)

In [10]:
# Calculate pts per 36 min by game
stats_dates_pos['pts_per36'] = 36 * (stats_dates_pos['pts']/stats_dates_pos['min'])

In [11]:
# Determine rolling_period on rolling metrics for entire analysis
rolling_period = 62

# Team Stats by Game/Date

In [19]:
stats_dates_pos.head()

,id,game_id,player_id,team_id,opponent_team_id,min,fga,fgm,fg_pct,fta,ftm,ft_pct,fg3a,fg3m,fg3_pct,oreb,dreb,reb,pts,ast,stl,blk,pf,turnover,fpts,game_date,player_position,pts_per36
26,7315084,473410,6,3,17,16.0,2,0,0.000,4,1,0.250,0,0,NaN,0,3,3,1,1,0,1,2,1,8.1,2021-10-19,F,2.250000
35,7315097,473410,8,17,3,28.0,10,3,0.300,1,1,1.000,10,3,0.30,0,4,4,10,6,2,0,0,0,29.8,2021-10-19,G,12.857143
48,7315095,473410,15,17,3,31.0,25,12,0.480,9,7,0.778,4,1,0.25,5,9,14,32,7,1,2,4,4,64.3,2021-10-19,F,37.161290
6,7315609,473409,17,14,10,26.0,9,3,0.333,2,1,0.500,4,2,0.50,0,4,4,9,2,0,0,2,1,15.8,2021-10-19,F,12.461538
9,7315607,473409,36,14,10,31.0,9,3,0.333,0,0,NaN,8,2,0.25,0,2,2,8,0,0,1,4,1,12.4,2021-10-19,G,9.290323


In [20]:
team_stats_by_game_pos = stats_dates_pos\
                        .groupby(['opponent_team_id','game_date','player_position'])\
                        [['pts']]\
                        .sum()\
                        .reset_index()

In [21]:
def team_pts_rolling_mean(group):
    group_indexed = group.set_index('game_date')
    
    rolling_group = group_indexed.groupby(['opponent_team_id','player_position'])\
                    [['pts']]\
                    .rolling(rolling_period,
                             min_periods = 1)\
                    .mean()\
                    .shift(1)\
                    .rename(columns = {'pts':'pgm'})

    final_rolling = rolling_group.droplevel([0,1])
    
    return final_rolling

In [22]:
team_stats_rolling_eff = team_stats_by_game_pos.groupby(['opponent_team_id','player_position'])\
                            .apply(team_pts_rolling_mean)\
                            .reset_index()

In [23]:
team_stats_rolling_eff.head()

,opponent_team_id,player_position,game_date,pgm
0,1,C,2021-10-21,NaN
1,1,C,2021-10-23,2.00
2,1,C,2021-10-25,6.50
3,1,C,2021-10-27,5.00
4,1,C,2021-10-30,9.75


# Player Stats by Game/Date

In [101]:
def player_pts_rolling_sum(group):
    group_indexed = group.sort_values('game_date')\
                        .set_index('game_date')
    
    rolling_group = group_indexed.groupby('player_id')\
                    [['min','pts']]\
                    .rolling(rolling_period,
                             min_periods = 1)\
                    .sum()

    final_rolling = rolling_group.droplevel(0)
    
    return final_rolling

In [102]:
player_stats_rolling_eff = stats_dates.groupby(stats_dates['player_id'])\
                            .apply(player_pts_rolling_sum)

# player_stats_rolling_eff = stats_dates.set_index('game_date')\
#                             .groupby(stats_dates['player_id'])\
#                             [['min','pts']]\
#                             .rolling(rolling_period,
#                                     min_periods = 1)\
#                             .sum()\
#                             .shift(1)\
#                             .dropna(how = 'all')

In [103]:
stat_categories = player_stats_rolling_eff.columns.drop('min')

In [104]:
for cat in stat_categories:
    player_stats_rolling_eff[cat+'_per36'] = player_stats_rolling_eff[cat] * (36/player_stats_rolling_eff['min'])

In [107]:
actual_eff_merged = pd.merge(stats_dates_pos[['game_date','player_id','player_position',
                                              'min','pts']],
                             player_stats_rolling_eff[['pts_per36']],
                             how = 'left',
                             left_on = ['player_id','game_date'],
                             right_index = True)#\
                    #.sort_values(['player_id','game_date'])

In [108]:
actual_eff_merged

,game_date,player_id,player_position,min,pts,pts_per36
26,2021-10-19,6,F,16.0,1,2.250000
35,2021-10-19,8,G,28.0,10,12.857143
48,2021-10-19,15,F,31.0,32,37.161290
6,2021-10-19,17,F,26.0,9,12.461538
9,2021-10-19,36,G,31.0,8,9.290323
...,...,...,...,...,...,...
96762,2024-01-29,56677866,F,12.0,0,9.818182
96659,2024-01-29,56677871,F,0.0,0,2.769231
96906,2024-01-29,56677872,G,0.0,0,16.095032
96514,2024-01-29,56783340,C,4.0,0,18.293160


In [18]:
per36_stat_cols = [i for i in actual_eff_merged.columns if ('_per36' in i)]

# League Position Average

In [24]:
def league_pos_pts_rolling_sum(group):
    group_game_day = group.groupby(['game_date','player_position'])\
                        [['min','pts']]\
                        .sum()\
                        .reset_index()
    
    group_indexed = group_game_day.sort_values('game_date')\
                        .set_index('game_date')
    
    rolling_group = group_indexed.groupby(['player_position'])\
                    [['min','pts']]\
                    .rolling(rolling_period,
                             min_periods = 1)\
                    .sum()\
                    .shift(1)

    final_rolling = rolling_group.droplevel([0])
    
    return final_rolling

In [25]:
league_pos_rolling_sum = stats_dates_pos.groupby(['player_position'])\
                            .apply(league_pos_pts_rolling_sum)\
                            .reset_index()

In [26]:
league_pos_rolling_sum['pts_per36'] = (league_pos_rolling_sum['pts']/league_pos_rolling_sum['min']) * 36

In [27]:
league_pos_rolling_sum

,player_position,game_date,min,pts,pts_per36
0,C,2021-10-19,NaN,NaN,NaN
1,C,2021-10-20,54.0,15.0,10.000000
2,C,2021-10-21,575.0,269.0,16.841739
3,C,2021-10-22,663.0,327.0,17.755656
4,C,2021-10-23,1107.0,526.0,17.105691
...,...,...,...,...,...
2509,UNK,2024-01-25,1995.0,716.0,12.920301
2510,UNK,2024-01-26,2009.0,733.0,13.134893
2511,UNK,2024-01-27,1973.0,713.0,13.009630
2512,UNK,2024-01-28,2001.0,727.0,13.079460


In [28]:
league_pos_rolling_sum[(league_pos_rolling_sum['player_position'] == 'C')
                        ]

,player_position,game_date,min,pts,pts_per36
0,C,2021-10-19,NaN,NaN,NaN
1,C,2021-10-20,54.0,15.0,10.000000
2,C,2021-10-21,575.0,269.0,16.841739
3,C,2021-10-22,663.0,327.0,17.755656
4,C,2021-10-23,1107.0,526.0,17.105691
...,...,...,...,...,...
417,C,2024-01-25,19004.0,8568.0,16.230688
418,C,2024-01-26,18886.0,8532.0,16.263476
419,C,2024-01-27,18940.0,8525.0,16.203801
420,C,2024-01-28,18658.0,8442.0,16.288563


# League Position Standard Deviation

In [62]:
def league_pos_pts_rolling_std(group):
    group_indexed = group.sort_values('game_date')\
                        .set_index('game_date')
    
    rolling_group = group_indexed.groupby(['player_position'])\
                    [['pts_per36']]\
                    .rolling(rolling_period,
                             min_periods = 1)\
                    .std()\
                    .shift(1)

    final_rolling = rolling_group.droplevel([0])
    
    return final_rolling

In [65]:
league_pos_rolling_std = stats_dates_pos.groupby(['player_position'])\
                            .apply(league_pos_pts_rolling_std)\
                            .reset_index()

In [73]:
league_pos_rolling_std.groupby(['player_position','game_date']).last().shift(1)

pts_per36
player_position game_date            
C               2021-10-19        NaN
                2021-10-20   5.874426
                2021-10-21   8.074736
                2021-10-22   9.092027
                2021-10-23   8.510522
...                               ...
UNK             2024-01-25   8.860995
                2024-01-26  11.596876
                2024-01-27  11.645064
                2024-01-28  11.352518
                2024-01-29  11.741385

[2514 rows x 1 columns]

In [100]:
stats_dates_pos[(stats_dates_pos['game_date'] == '2021-10-20')
               & (stats_dates_pos['player_position'] == 'C')
               & (stats_dates_pos['min'].notna())]#['pts_per36'].mean()

,id,game_id,player_id,team_id,opponent_team_id,min,fga,fgm,fg_pct,fta,ftm,ft_pct,fg3a,fg3m,fg3_pct,oreb,dreb,reb,pts,ast,stl,blk,pf,turnover,fpts,game_date,player_position,pts_per36
107,7322581,473415,3,15,6,32.0,7,4,0.571,0,0,NaN,0,0,NaN,6,8,14,8,3,1,1,0,2,33.3,2021-10-20,C,9.000000
58,7322566,473415,9,6,15,29.0,11,11,1.000,4,3,0.750,0,0,NaN,1,3,4,25,1,3,3,1,1,48.3,2021-10-20,C,31.034483
174,7325298,473421,22,24,8,33.0,10,6,0.600,2,2,1.000,3,1,0.333,2,4,6,15,2,1,0,2,2,26.2,2021-10-20,C,16.363636
286,7323219,473418,28,22,27,27.0,8,6,0.750,4,3,0.750,4,3,0.750,3,1,4,18,4,1,4,5,3,40.8,2021-10-20,C,24.000000
85,7322121,473413,46,28,30,26.0,4,2,0.500,6,5,0.833,0,0,NaN,6,3,9,9,0,0,0,2,2,17.8,2021-10-20,C,12.461538
322,7322214,473412,60,5,9,0.0,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,2021-10-20,C,NaN
53,7322569,473415,118,6,15,0.0,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,2021-10-20,C,NaN
115,7322485,473417,137,23,19,19.0,4,2,0.500,3,2,0.667,0,0,NaN,2,15,17,6,3,2,2,2,4,38.9,2021-10-20,C,11.368421
357,7323939,473419,176,29,21,29.0,9,6,0.667,6,4,0.667,0,0,NaN,5,16,21,16,0,0,1,1,1,43.2,2021-10-20,C,19.862069
239,7322500,473417,207,19,23,0.0,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,2021-10-20,C,NaN


In [87]:
actual_eff_merged.sort_values('game_date',
                             inplace = True)

In [109]:
actual_eff_merged[(actual_eff_merged['player_position'] == 'C')
                 & (actual_eff_merged['pts_per36'].notna())]

,game_date,player_id,player_position,min,pts,pts_per36
4,2021-10-19,220,C,13.0,5,13.846154
10,2021-10-19,250,C,13.0,2,5.538462
37,2021-10-19,283,C,28.0,8,10.285714
107,2021-10-20,3,C,32.0,8,9.000000
58,2021-10-20,9,C,29.0,25,31.034483
...,...,...,...,...,...,...
96585,2024-01-29,39398582,C,0.0,0,12.093750
96516,2024-01-29,39613724,C,0.0,0,4.666667
96590,2024-01-29,56677738,C,0.0,0,18.000000
96809,2024-01-29,56677840,C,33.0,16,12.712743


In [123]:
league_pos_rolling_std = actual_eff_merged.set_index('game_date')\
                            .groupby('player_position')\
                            [['pts_per36']]\
                            .rolling(rolling_period,
                                    min_periods = 1)\
                            .std()\
                            .groupby(level = [0,1])\
                            .tail(1)\
                            .reset_index()

In [124]:
league_pos_rolling_std

,player_position,game_date,pts_per36
0,C,2021-10-19,4.167951
1,C,2021-10-20,7.949697
2,C,2021-10-21,9.029786
3,C,2021-10-22,8.249702
4,C,2021-10-23,8.379770
...,...,...,...
2509,UNK,2024-01-25,2.199533
2510,UNK,2024-01-26,2.225209
2511,UNK,2024-01-27,2.201393
2512,UNK,2024-01-28,2.236069


In [125]:
league_pos_rolling_std[league_pos_rolling_std['player_position'] == 'C']

,player_position,game_date,pts_per36
0,C,2021-10-19,4.167951
1,C,2021-10-20,7.949697
2,C,2021-10-21,9.029786
3,C,2021-10-22,8.249702
4,C,2021-10-23,8.379770
...,...,...,...
417,C,2024-01-25,5.198412
418,C,2024-01-26,4.885941
419,C,2024-01-27,4.335266
420,C,2024-01-28,4.696235


In [37]:
actual_eff_merged.set_index('game_date')\
    [['pts_per36']]\
    .groupby(level = 0)\
    .rolling(rolling_period,
            min_periods = 1)\
    .std()

pts_per36
game_date  game_date            
2021-10-19 2021-10-19        NaN
           2021-10-19        NaN
           2021-10-19        NaN
           2021-10-19        NaN
           2021-10-19        NaN
...                          ...
2024-01-29 2024-01-29   5.116612
           2024-01-29   5.086901
           2024-01-29   5.101550
           2024-01-29   5.099315
           2024-01-29   5.119354

[96937 rows x 1 columns]

In [ ]:
    group['pts_per36'] = 36 * (group['pts']/group['min'])
    
    group_per_indexed = group.sort_values('game_date')\
                            .set_index('game_date')
    
    rolling_group = group_per_indexed.groupby('player_position')\
                        [['pts_per36']]\
                        .rolling(rolling_period,
                                min_periods = 1)\
                        .agg(['mean','std'])\
                        .shift(1)
    
    final_rolling = rolling_group.droplevel([0])

# Player Stat Standardization

In [ ]:
actual_eff_merged.head()

In [ ]:
league_pos_rolling_sum.head()

In [ ]:
rel_stats = stats_dates_pos[['game_date',
                             'player_id','player_position',
                             'team_id','opponent_team_id',
                             'min','pts']]

In [ ]:
player_eff_stats = pd.merge(rel_stats,
                            actual_eff_merged[['game_date','player_id',
                                               'pts_per36']]\
                               .rename(columns = {'pts_per36':'pts_per36_before_game'}),
                            how = 'left',
                            left_on = ['game_date','player_id'],
                            right_on = ['game_date','player_id']
                           )

In [ ]:
team_stats_rolling_eff.head()

# Extra

In [ ]:
def league_team_pts_rolling_mean(group):
    group_indexed = group.set_index('game_date')
    
    rolling_group = group_indexed.groupby(['opponent_team_id','player_position'])\
                    [['pts']]\
                    .rolling(rolling_period,
                             min_periods = 1)\
                    .mean()\
                    .shift(1)\
                    .rename(columns = {'pts':'pgm'})

    final_rolling = rolling_group.droplevel([0,1])
    
    return final_rolling

In [ ]:
team_stats_rolling_eff[(team_stats_rolling_eff['opponent_team_id'] == 1)
                       & (team_stats_rolling_eff['player_position'] == 'C')]

In [ ]:
team_stats_by_game[team_stats_by_game['team_id'] == 10]